In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



In [2]:
df1=pd.read_table("/kaggle/input/wikiqa-corpus/WikiQA.tsv")
df2=pd.read_table("/kaggle/input/wikiqa-corpus/WikiQASent.pos.ans.tsv")
df3=pd.read_table("/kaggle/input/wikiqa-corpus/WikiQA-train.tsv")
df4=pd.read_table("/kaggle/input/wikiqa-corpus/WikiQA-test.tsv")
df5=pd.read_table("/kaggle/input/wikiqa-corpus/WikiQA-dev.tsv")

In [3]:
df=pd.concat([df1,df3,df4,df5])

In [4]:
df.head()

,QuestionID,Question,DocumentID,DocumentTitle,SentenceID,Sentence,Label
0,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-0,African immigration to the United States refer...,0
1,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-1,The term African in the scope of this article ...,0
2,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-2,From the Immigration and Nationality Act of 19...,0
3,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-3,African immigrants in the United States come f...,0
4,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-4,"They include people from different national, l...",0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58404 entries, 0 to 2732
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   QuestionID     58404 non-null  object
 1   Question       58404 non-null  object
 2   DocumentID     58404 non-null  object
 3   DocumentTitle  58404 non-null  object
 4   SentenceID     58404 non-null  object
 5   Sentence       58404 non-null  object
 6   Label          58404 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 3.6+ MB


In [6]:
df=df.drop(["DocumentID","Label","QuestionID","SentenceID"],axis =1)

In [7]:
df.isnull().sum()

Question         0
DocumentTitle    0
Sentence         0
dtype: int64

In [8]:
pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=8175753f15576cd7a99a24a2224099a66215ff807a49d93d38239f5e8f9be1b7
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers
Note: you may need to restart the kernel to use updated packages.


In [9]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
from progressbar import ProgressBar
pbar = ProgressBar()
question_list = []
for i,row in pbar(df.iterrows()):
    question_list.append(
      (
        
        model.encode(row['Question']).tolist(),
        {
            'Title': row['DocumentTitle'],
            'Answer': row['Sentence']
        }
      )
  )
    

In [20]:
import json
json_data = json.dumps(question_list)

# Write the JSON string to a file
with open("output.json", "w") as json_file:
    json_file.write(json_data)


In [22]:
!pip install sentence-transformers pinecone-client -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [66]:
import pinecone
pinecone.init(api_key='bc633d1d-54a8-4234-b24c-d5db4f5908a0', environment='asia-southeast1-gcp-free')


In [69]:
pinecone.create_index(name='question-search', dimension=1024)
index = pinecone.Index('question-search')

In [ ]:
index.upsert(vectors=question__list)

In [70]:
sentnce="About moon"
xq = model.encode([sentnce]).tolist()


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [65]:
from sklearn.metrics.pairwise import cosine_similarity
similar_products=[]
for i in range(len(question_list)):
        embedding=np.array(question_list[i][0])
        similarity = cosine_similarity(xq, embedding.reshape(1,-1))
        if similarity > 0.56:
            similar_products.append((question_list[i][1], similarity[0, 0]))
similar_products.sort(key=lambda x: x[1], reverse=True)
print(similar_products)

[({'Title': 'Moon', 'Answer': 'Hubble views of Copernicus crater (inset), which is about 58 miles (93 km) wide'}, 0.5706752885570903), ({'Title': 'Moon', 'Answer': 'Red and orange tinted Moon, as seen from Earth during a lunar eclipse, where the Earth comes between the Moon and Sun'}, 0.5706752885570903), ({'Title': 'Moon', 'Answer': 'The Moon is the only natural satellite of the Earth , and the fifth largest satellite in the Solar System .'}, 0.5706752885570903), ({'Title': 'Moon', 'Answer': 'It is the largest natural satellite of a planet in the Solar System relative to the size of its primary , having 27% the diameter and 60% the density of Earth, resulting in its mass .'}, 0.5706752885570903), ({'Title': 'Moon', 'Answer': 'The Moon is the second densest satellite after Io , a satellite of Jupiter.'}, 0.5706752885570903), ({'Title': 'Moon', 'Answer': 'The Moon is in synchronous rotation with Earth, always showing the same face with its near side marked by dark volcanic maria that fi